In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.layers import Layer, Embedding, Dense, LayerNormalization, Dropout
import numpy as np

#load the harry potter book as the dataset ->  url - https://www.kaggle.com/datasets/shubhammaindola/harry-potter-books
def load_data(file_path):
  with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
  return text

file_path = "hp_1.txt"
text = load_data(file_path).lower()


# Tokenize the text
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1


# Convert text to sequences
input_sequences = []
tokens = tokenizer.texts_to_sequences([text])[0]
seq_length = 50

# First seq_length tokens (input): Used for training the model.
# Last token (target): Used as the label the model tries to predict.
# so total of (50 + 1) in one input_sequence index

for i in range(seq_length, len(tokens)):
    input_sequences.append(tokens[i - seq_length:i + 1])

#print(input_sequences[0])

# Pad sequences and split inputs/targets
# after this X will have inputs and y will have label for those inputs
input_sequences = np.array(pad_sequences(input_sequences, maxlen=seq_length + 1, padding='pre'))
X, y = input_sequences[:, :-1], input_sequences[:, -1]

# One-hot encode the labels , note- there are other ways for
# encoding like pre-trained word2vec encoding and so on
y = tf.keras.utils.to_categorical(y, num_classes=total_words)

## Core of the Transformer model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Embedding, Dense, LayerNormalization, Dropout

class MultiHeadAttention(Layer):

    def __init__(self, embed_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # example - 8

        self.embed_dim = embed_dim # example - 512
        # embed_dim = dimension of Q, K, and V before splitting into multiple heads
        # It is same as total dimension of the input embeddings (word embeddings)

        self.projection_dim = embed_dim // num_heads # Size of Each Attention Head's Subspace
        # Each head gets a smaller subspace of the embedding dimension
        # example - 64

        # Fully connected (dense) layers that project the input into Q,K,V
        # These layers map the input embeddings to the same embed_dim
        # These layers will be reshaped / split later to split across attention heads
        # A single large matrix multiplication is more efficient than many small ones
        # GPUs love large matrix multiplications because they are optimized for parallel computation
        # This allows TF/Keras to efficiently batch the computation, leveraging better GPU memory utilization

        self.query_dense = Dense(embed_dim) # Q Determines "what to focus on"
        self.key_dense = Dense(embed_dim) # K Acts as "labels" to be matched with queries
        self.value_dense = Dense(embed_dim) # V Holds the actual information

        self.combine_heads = Dense(embed_dim)
        # After multi-head attention is applied, the outputs from all heads are concatenated back into embed_dim


    def attention(self, query, key, value):
        scores = tf.matmul(query, key, transpose_b=True)
        scores /= tf.math.sqrt(tf.cast(self.projection_dim, tf.float32)) # converting integer to a float32 tensor

        attention_probs = tf.nn.softmax(scores, axis=-1) # how much attention each token should give to other tokens
        # The higher the score, the more focus that token gets
        # Softmax should be applied along the keys (i.e., across the last dimension of the scores matrix)
        # Each row corresponds to a query token attending to all key tokens
        # This ensures that each query distributes its attention across all keys properly
        # Each row sums to 1

        return tf.matmul(attention_probs, value), attention_probs

    # x - query, key or value with shape - (batch_size, seq_len, embed_dim)
    # batch_size - number of sequences being processed in parallel (for batch processing)
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
        # before transpose - (batch_size, seq_len, num_heads, projection_dim)
        # after transpose - (batch_size, num_heads, seq_len, projection_dim)
        # The -1 in tf.reshape is a placeholder that tells TensorFlow to automatically
        # infer that dimension's value based on the total number of elements in the tensor
        # -1 is replaced by seq_len by tensorflow

    # In TF,Keras - call(self, inputs) is a standard method used inside Layer subclasses
    # to define the forward pass of a neural network layer

    def call(self, inputs):
        query, key, value = inputs
        batch_size = tf.shape(query)[0] # (batch_size, seq_len, embed_dim)

        query = self.split_heads(self.query_dense(query), batch_size)
        key = self.split_heads(self.key_dense(key), batch_size)
        value = self.split_heads(self.value_dense(value), batch_size)

        attention, _ = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        # before transpose - (batch_size, num_heads, seq_len, projection_dim)
        # after transpose -  (batch_size, seq_len, num_heads, projection_dim)

        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))
        # Merges all heads back into a single vector
        # (batch_size, seq_len, num_heads, projection_dim) → (batch_size, seq_len, embed_dim)

        return self.combine_heads(concat_attention)

class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim),
        ])
        # y = (x - mean) / root(variance + epsilon)
        # epsilon ensures we never divide by zero
        # it is small enough not to affect the result but large enough to prevent instability
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att([inputs, inputs, inputs])

        # Dropout randomly deactivates some neurons during training to reduce overfitting
        # Ensure dropout is only applied during training, not inference
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output) # Residual Connection
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output) # Residual Connection

class TokenAndPositionEmbedding(Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)
        # The Embedding layer takes an integer tensor and replaces each integer with an embed_dim-sized vector
        # example - positions = [0, 1, 2, 3]
        # after embedding - positions = [
        #   [0.2, 0.1, 0.3, 0.5, 0.6, 0.9, 0.7, 0.8],  # Position 0
        #   [0.4, 0.2, 0.1, 0.6, 0.5, 0.7, 0.9, 0.3],  # Position 1
        #   [0.5, 0.3, 0.8, 0.2, 0.7, 0.4, 0.6, 0.1],  # Position 2
        #   [0.9, 0.6, 0.2, 0.3, 0.1, 0.8, 0.4, 0.7]   # Position 3
        # ]

        # initial shape of x - (batch_size, seq_len)
        # batch_size: Number of sentences in a batch
        # seq_len: Number of tokens (words) in each sentence
        # Each value in x is an integer index from 0 to vocab_size - 1
        # after embedding - (batch_size, seq_len, embed_dim)

        # example - embed_dim = 8, batch_size = 2
        #     x = [
        #   [ [0.2, 0.1, 0.4, 0.3, 0.8, 0.7, 0.6, 0.9],  # Token 2
        #     [0.5, 0.3, 0.9, 0.1, 0.2, 0.6, 0.8, 0.7],  # Token 5
        #     [0.4, 0.9, 0.2, 0.3, 0.1, 0.7, 0.5, 0.6],  # Token 1
        #     [0.3, 0.8, 0.6, 0.2, 0.5, 0.9, 0.7, 0.4]   # Token 7
        #   ],  # First sentence

        #   [ [0.1, 0.6, 0.9, 0.7, 0.3, 0.5, 0.2, 0.8],  # Token 0
        #     [0.4, 0.2, 0.3, 0.9, 0.7, 0.5, 0.1, 0.6],  # Token 3
        #     [0.8, 0.5, 0.4, 0.1, 0.6, 0.3, 0.2, 0.7],  # Token 8
        #     [0.9, 0.3, 0.5, 0.7, 0.8, 0.2, 0.6, 0.1]   # Token 4
        #   ]   # Second sentence
        # ]


    def call(self, x):
        # the maximum sequence length the model can handle
        maxlen = tf.shape(x)[-1] # sets maxlen to the length of the input sequence
        positions = tf.range(start=0, limit=maxlen, delta=1) # Generate [0, 1, 2, ..., maxlen-1]
        positions = self.pos_emb(positions) # Each position index is mapped to a trainable embedding of shape (maxlen, embed_dim)
        x = self.token_emb(x) # Each token ID in x is mapped to an embedding of shape (batch_size, maxlen, embed_dim)
        return x + positions

        # x has shape (batch_size, seq_len, embed_dim)
        # positions has shape (maxlen, embed_dim)
        # But maxlen == seq_len, so positions effectively has shape (seq_len, embed_dim).
        # TensorFlow broadcasts positions across batch_size, treating it as if it were (1, seq_len, embed_dim).
        # This allows element-wise addition between x and position

## Model the whole architecture , compile and run the training

In [ ]:
# Model Parameters
embed_dim = 128  # Embedding size
num_heads = 4    # Number of attention heads
ff_dim = 512     # Feed-forward layer size
maxlen = seq_length # here it is 50 defined above

# below total words = 6662 (see above - basically all tokens in the text)

# Build the model
inputs = tf.keras.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, total_words, embed_dim)
x = embedding_layer(inputs)
print(x.shape)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x, training=True)
print(x.shape)
x = x[:, -1, :]
print(x.shape)
x = Dense(total_words, activation="softmax")(x)
print(x.shape)
model = tf.keras.Model(inputs=inputs, outputs=x)

# Compile the model
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

(None, 50, 128)
(None, 50, 128)
(None, 128)
(None, 6663)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ token_and_position_embedding         │ (None, 50, 128)             │         859,264 │
│ (TokenAndPositionEmbedding)          │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_block (TransformerBlock) │ (None, 50, 128)             │         198,272 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ get_item (GetItem)                   │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 6663)                │         859,527 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,917,063 (7.31 MB)

 Trainable params: 1,917,063 (7.31 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(X, y, batch_size=32, epochs=10)

Epoch 1/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 283s 110ms/step - accuracy: 0.0820 - loss: 6.5049
Epoch 2/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 321s 110ms/step - accuracy: 0.1595 - loss: 5.0777
Epoch 3/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 322s 110ms/step - accuracy: 0.2095 - loss: 4.2903
Epoch 4/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 322s 110ms/step - accuracy: 0.2578 - loss: 3.6334
Epoch 5/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 278s 110ms/step - accuracy: 0.3177 - loss: 3.1068
Epoch 6/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 319s 109ms/step - accuracy: 0.3962 - loss: 2.6140
Epoch 7/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 324s 109ms/step - accuracy: 0.4798 - loss: 2.1751
Epoch 8/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 325s 111ms/step - accuracy: 0.5539 - loss: 1.8186
Epoch 9/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 321s 111ms/step - accuracy: 0.6135 - loss: 1.5152
Epoch 10/10
2531/2531 ━━━━━━━━━━━━━━━━━━━━ 320s 110ms/step - accuracy: 0.6653 - loss: 1.2756


In [ ]:
def generate_text(seed_text, next_words, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word = tokenizer.index_word[np.argmax(predicted)]
        seed_text += " " + predicted_word
    return seed_text

# Generate text
seed_text = "harry looked at"
generated_text = generate_text(seed_text, next_words=50, max_sequence_len=seq_length + 1)
print(len(generated_text))

285


In [ ]:
print(generated_text)

harry looked at the first time he got some wizarding families what’s the matter with him without it in the air ” said harry ron and hermione were in danger he himself darted around the board taking almost as many white pieces as they walked off the board leaving three sparkling with a


What Is Missing Compared to ChatGPT?

- Masked Attention:

ChatGPT uses causal masking so that a word cannot see future words during training. Our model uses regular attention, which allows it to see the entire sequence.

- Multiple Stacked Transformer Blocks:

ChatGPT has many layers (e.g., 12, 24, 96 layers). Our model has only one Transformer block.

- Tokenization & Byte-Pair Encoding (BPE):

ChatGPT does not use simple tokenization; it uses Byte-Pair Encoding (BPE) or WordPiece for better vocabulary handling. Our model uses basic word tokenization.

- Training on Large Datasets:

ChatGPT is trained on hundreds of GBs of text. Our model is trained on a single Harry Potter book (very limited).

- Decoding Strategies for Text Generation:

ChatGPT uses sampling (top-k, nucleus sampling) or beam search to generate text. Our model does not have a decoding strategy.